# Isoform protein identification
 The purpose of this analysis is to extract information on the number of genes with multiple isoforms
### Mass-spec information only
- Number of genes where isoforms were only identified with shared peptides
- Number of genes where a single isoform was identified with unique peptides
- Number of genes where multiple isoforms were identified with unique peptides


### Transcript incorporation mass spec
- Number of genes where a unique isoform was identified with PacBio, but not identified with GENCODE
- Number of genes where an isoform that would otherwise be discarded was rescued
- Number of genes where ambigous protein groups have multiple confirmed isoforms due to transcript abundance



In [1]:
import pandas as pd
from huvec_analysis import data_loader, huvec_config
from Bio import SeqIO

import os
# all statistics go into a directory
if not os.path.exists('stats'):
    os.makedirs('stats')


In [2]:

protein_sequences = [] 
for record in SeqIO.parse(f'{huvec_config.PIPELINE_RESULTS_DIRECTORY}/hybrid_protein_database/huvec_hybrid.fasta', 'fasta'):
    acc = record.description.split('|')[1].strip()
    gene_name = record.description.split('GN=')[1]
    val = {
        'accession': acc, 
        'gene' : gene_name,
        'sequence': str(record.seq)
    }
    protein_sequences.append(val)

transcript_abundance = pd.read_table(f'{huvec_config.PIPELINE_RESULTS_DIRECTORY}/hybrid_protein_database/huvec_refined_high_confidence.tsv', usecols=['base_acc', 'CPM'])


In [3]:
huvec_peptides = data_loader.read_peptide_file(data_loader.pacbio_hybrid_peptide_file, data_loader.hybrid_gene_map)
huvec_peptides['is_high_confidence'] =huvec_peptides['accs'].apply(lambda accs: data_loader.is_high_confidence(accs, data_loader.accs_in_hiconf_space))

# huvec_peptides['is_isoform_distinct'] = huvec_peptides['accs'].apply(lambda accs: len(accs) == 1)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/bj8th/opt/anaconda3/envs/lrpg/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-ff12fd93d46e>", line 1, in <module>
    huvec_peptides = data_loader.read_peptide_file(data_loader.pacbio_hybrid_peptide_file, data_loader.hybrid_gene_map)
  File "/Users/bj8th/Documents/Sheynkman-Lab/GitHub/HUVEC_manuscript_analysis/huvec_analysis/data_loader.py", line 49, in read_peptide_file
    return read_metamorpheus_file(peptides_file, gene_map)
  File "/Users/bj8th/Documents/Sheynkman-Lab/GitHub/HUVEC_manuscript_analysis/huvec_analysis/data_loader.py", line 41, in read_metamorpheus_file
    metamorpheus_table = pd.read_table(metamorpheus_file)
  File "/Users/bj8th/opt/anaconda3/envs/lrpg/lib/python3.8/site-packages/pandas/io/parsers.py", line 689, in read_table
    return _read(filepath_or_buffer, kwds)
  File "/Users/bj8th/opt/anaconda3

In [ ]:
def find_accessions_and_genes(peptide, protein_sequences):
    accessions = []
    genes = []
    if '|' in peptide:
        all_peptides = set(peptide.split('|'))
        for val in protein_sequences:
            for pep in all_peptides:
                if pep in val['sequence']:
                    accessions.append(val['accession'])
                    genes.append(val['gene'])
                    continue
    else:
        for val in protein_sequences:
            if peptide in val['sequence']:
                accessions.append(val['accession'])
                genes.append(val['gene'])
    return [accessions, genes]



#huvec_peptides = huvec_peptides.head().copy()
huvec_peptides['accessions_genes'] = huvec_peptides['Base Sequence'].apply(lambda peptide: find_accessions_and_genes(peptide, protein_sequences))
huvec_peptides['accessions'] = huvec_peptides['accessions_genes'].apply(lambda x : x[0])
huvec_peptides['genes'] = huvec_peptides['accessions_genes'].apply(lambda x: x[1])
huvec_peptides['num_accessions'] = huvec_peptides['accessions'].apply(len)
huvec_peptides['is_peptide_uniquely_mapping'] = huvec_peptides['num_accessions'] == 1 

huvec_peptides['is_isoform_distinct'] = huvec_peptides['is_peptide_uniquely_mapping']



## Make peptide table
Peptide table contains columns in AllPeptides.tsv
- filtered to only include target peptides
- filtered to only include peptides with QValue < 0.01
- map peptide Base Sequence to fasta database


In [ ]:
huvec_peptides['accessions'] = huvec_peptides['accessions_genes'].apply(lambda x : x[0])
huvec_peptides['genes'] = huvec_peptides['accessions_genes'].apply(lambda x: x[1])
huvec_peptides['num_accessions'] = huvec_peptides['accessions'].apply(len)
huvec_peptides['is_peptide_uniquely_mapping'] = huvec_peptides['num_accessions'] == 1 

huvec_peptides['is_isoform_distinct'] = huvec_peptides['is_peptide_uniquely_mapping']
huvec_peptides['accessions'] = huvec_peptides['accessions'].apply(lambda x: '|'.join(x))
huvec_peptides['genes'] = huvec_peptides['genes'].apply(lambda x: '|'.join(x))
huvec_peptides.to_csv('./stats/huvec_peptides_mapped_accessions.tsv', sep='\t', index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/bj8th/opt/anaconda3/envs/lrpg/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1695ab7550b2>", line 9, in <module>
    huvec_peptides.to_csv('./stats/huvec_peptides_mapped_accessions.tsv', sep='\t', index=False)
  File "/Users/bj8th/opt/anaconda3/envs/lrpg/lib/python3.8/site-packages/pandas/core/generic.py", line 3387, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "/Users/bj8th/opt/anaconda3/envs/lrpg/lib/python3.8/site-packages/pandas/io/formats/format.py", line 1083, in to_csv
    csv_formatter.save()
  File "/Users/bj8th/opt/anaconda3/envs/lrpg/lib/python3.8/site-packages/pandas/io/formats/csvs.py", line 228, in save
    with get_handle(
  File "/Users/bj8th/opt/anaconda3/envs/lrpg/lib/python3.8/site-packages/pandas/io/common.py", line 642, in get_handle
    handle = open(
TimeoutError: [Errno 60] Ope

In [ ]:



huvec_peptides = pd.read_table('./stats/huvec_peptides_mapped_accessions.tsv')
huvec_peptides['accessions'] = huvec_peptides['accessions'].apply(lambda x: str(x).split('|'))
huvec_peptides['genes'] = huvec_peptides['genes'].apply(lambda x: str(x).split('|'))

huvec_peptides['accessions_genes'] = huvec_peptides.apply(lambda row: [row['accessions'], row['genes']] ,axis=1)
huvec_peptides['num_accessions'] = huvec_peptides['accessions'].apply(len)
huvec_peptides['is_peptide_uniquely_mapping'] = huvec_peptides['num_accessions'] == 1 

huvec_peptides['is_isoform_distinct'] = huvec_peptides['is_peptide_uniquely_mapping']



In [ ]:
huvec_peptides = pd.read_table('./stats/huvec_peptides_mapped_accessions.tsv')
huvec_peptides['accessions'] = huvec_peptides['accessions'].apply(lambda x: str(x).split('|'))
huvec_peptides['genes'] = huvec_peptides['genes'].apply(lambda x: str(x).split('|'))

huvec_peptides['accessions_genes'] = huvec_peptides.apply(lambda row: [row['accessions'], row['genes']] ,axis=1)



,File Name,Scan Number,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,...,PEP,PEP_QValue,accs,genes,is_high_confidence,accessions_genes,accessions,num_accessions,is_peptide_uniquely_mapping,is_isoform_distinct
2,"210827_HUVEC5,6_tryp_HCDorbi_4hr_Fr11-calib",92300,106.9019,200.0,407915360.0,92257,3.0,1251.59902,3751.77522,41.289,...,0.00004,0.000028,"['PB.974.2', 'PB.974.21', 'PB.974.32', 'PB.974...","[LMNA, LMNA, LMNA, LMNA, LMNA]",True,"[[PB.974.2, PB.974.21, PB.974.31, PB.974.32, P...","[PB.974.2, PB.974.21, PB.974.31, PB.974.32, PB...",5,False,False


### Make isoform table
The isoform table contains information of the unique and shared peptides identified through mass-spectrometry for each isoform. Also included is the transcript abundance for PacBio isoforms

In [ ]:

huvec_peptides['num_isoform_multimap'] = huvec_peptides['accessions'].apply(lambda x: [len(x)] * len(x))
huvec_peptides['Sequence'] = huvec_peptides['Base Sequence']
exploded = huvec_peptides.set_index(['Sequence'])[['accessions', 'genes', 'num_isoform_multimap']].apply(pd.Series.explode).reset_index()
isoform_unique_peptides = exploded[exploded['num_isoform_multimap'] == 1].groupby(['accessions', 'genes'])['Sequence'].apply(list).reset_index()
isoform_unique_peptides.rename(columns = {'Sequence' : 'unique_peptides'}, inplace=True)
isoform_shared_peptides = exploded[exploded['num_isoform_multimap'] > 1].groupby(['accessions', 'genes'])['Sequence'].apply(list).reset_index()
isoform_shared_peptides.rename(columns = {'Sequence' : 'shared_peptides'}, inplace=True)

huvec_isoforms = isoform_unique_peptides.merge(isoform_shared_peptides, how = 'outer', on = ['accessions','genes'])
huvec_isoforms['is_confirmed_ms'] = ~huvec_isoforms['unique_peptides'].isna()
huvec_isoforms.rename(columns={'accessions' : 'accession', 'genes': 'gene'}, inplace=True)
huvec_isoforms['unique peptides'] = huvec_isoforms['unique_peptides'].apply(lambda x: '|'.join(x) if type(x) is list else '')
huvec_isoforms['shared peptides'] = huvec_isoforms['shared_peptides'].apply(lambda x: '|'.join(x) if type(x) is list else '')
huvec_isoforms = huvec_isoforms.merge(transcript_abundance[['base_acc', 'CPM']], left_on='accession', right_on='base_acc', how = 'left')

huvec_isoforms.drop(columns=['unique_peptides', 'shared_peptides', 'base_acc'], inplace=True)
huvec_isoforms.to_csv('./stats/huvec_isoforms_with_found_peptides.tsv', sep='\t', index=False)

## Make gene table
The gene table contains information on isoforms confirmed with mass spec as well as isoforms that only contain shared peptides

In [ ]:
confirmed_isoforms = huvec_isoforms[huvec_isoforms['is_confirmed_ms']][['accession', 'gene']]
confirmed_gene_isoforms = confirmed_isoforms.groupby('gene')['accession'].apply(list).reset_index(name = 'isoforms_with_unique_peptides')
confirmed_gene_isoforms['number_of_isoforms_with_unique_peptides'] = confirmed_gene_isoforms['isoforms_with_unique_peptides'].apply(len)
confirmed_gene_isoforms['isoforms_with_unique_peptides'] = confirmed_gene_isoforms['isoforms_with_unique_peptides'].apply(lambda x: '|'.join(x))

shared_isoforms = huvec_isoforms[~huvec_isoforms['is_confirmed_ms']][['accession', 'gene']]
shared_gene_isoforms = shared_isoforms.groupby('gene')['accession'].apply(list).reset_index(name = 'isoforms_with_only_shared_peptides')
shared_gene_isoforms['number_of_isoforms_with_shared_only_peptides'] = shared_gene_isoforms['isoforms_with_only_shared_peptides'].apply(len)
shared_gene_isoforms['isoforms_with_only_shared_peptides'] = shared_gene_isoforms['isoforms_with_only_shared_peptides'].apply(lambda x: '|'.join(x))


huvec_genes = pd.merge(confirmed_gene_isoforms, shared_gene_isoforms, on='gene', how = 'outer')
huvec_genes['isoforms_with_unique_peptides'].fillna('', inplace=True)
huvec_genes['isoforms_with_only_shared_peptides'].fillna('', inplace=True)
huvec_genes['number_of_isoforms_with_unique_peptides'].fillna(0, inplace=True)
huvec_genes['number_of_isoforms_with_shared_only_peptides'].fillna(0, inplace=True)
huvec_genes = huvec_genes.astype({"number_of_isoforms_with_unique_peptides":'int', "number_of_isoforms_with_shared_only_peptides":'int'}) 
isoforms_in_database = pd.DataFrame(protein_sequences)
genes_database_sizes = isoforms_in_database.groupby('gene').size().reset_index(name='number_isoforms_in_database')
huvec_genes = huvec_genes.merge(genes_database_sizes, on = 'gene', how = 'left' )
huvec_genes.to_csv('./stats/huvec_genes_found_in_mass_spec.tsv', sep='\t', index=False)


## Breakdown of genes found in mass spec

In [ ]:
print("Number of genes found through mass spec")
print(len(huvec_genes))


Number of genes found through mass spec
10583


In [ ]:
print('All genes: PB high confidence + Gencode')
print('The number of genes where there are no unique peptides mapping to any isoform')
print(len(huvec_genes[huvec_genes['isoforms_with_unique_peptides'] == '']))


All genes: PB high confidence + Gencode
The number of genes where there are no unique peptides mapping to any isoform
6112


In [ ]:
print("Number of genes with at least one unique isoform")
print(len(
    huvec_genes[
        (huvec_genes['number_of_isoforms_with_unique_peptides'] > 0)]
    ))
print("Number of genes with confirmed isoforms where only one isoform is in database")
print(len(
    huvec_genes[
        (huvec_genes['number_of_isoforms_with_unique_peptides'] > 0) &
        (huvec_genes['number_isoforms_in_database'] == 1)]
    ))
print("Number of genes with unique isoforms where  multiple isoforms are in the database")
print(len(
    huvec_genes[
        (huvec_genes['number_of_isoforms_with_unique_peptides'] > 0) &
        (huvec_genes['number_isoforms_in_database']  > 1)]
    ))



Number of genes with at least one unique isoform
4471
Number of genes with confirmed isoforms where only one isoform is in database
1758
Number of genes with unique isoforms where  multiple isoforms are in the database
2713


In [ ]:
print("Breakdown of number of isoforms confirmed per gene")
huvec_genes.groupby('number_of_isoforms_with_unique_peptides').size().reset_index(name='number_of_genes')

Breakdown of number of isoforms confirmed per gene


,number_of_isoforms_with_unique_peptides,number_of_genes
0,0,6112
1,1,4368
2,2,92
3,3,10
4,7,1
